# ch7. Generator

## 0. Discriptor Review
- refer link([blog1](https://m.blog.naver.com/PostView.naver?isHttpsRedirect=true&blogId=dudwo567890&logNo=130165713734))


- 디스크립터를 활용함으로써 강력한 추상화 가능
- 디스크립터는 재사용 가능한(반복 가능한) 프로퍼티 로직을 구할 때 좋음
  - 캡슐화

- 비즈니스 로직을 넣으면 안됨

> garbage collection 
- 사용하지 않는 메모리를 반환

### 참조(Reference)
- 생성된 객체에 대해서 여러 다른 이름으로 참조 할 때 참조카운터가 증가함
- 참조카운터가 0보다 큰 경우 가비지컬렉션이 일어나지 않음

### 약한 참조(Weak reference)
- 변수가 객체를 참조할 때, 참조카운터를 증가시키지 않음
- `weakref` 모듈 사용

In [1]:
import weakref
class C:
    a = 1

c = C()
r = weakref.ref(c)
r().a

1

In [2]:
p = weakref.proxy(c)
p.a

1

# 1. 제너레이터 만들기
- 고성능이면서 메모리를 적게 사용하는 반복을 위한 방법
- 이터러블을 생성해주는 객체(함수)
- 주요 목적은 **메모리 절약**
- 파이썬에서 어떤 함수라도 yield 키워드를 사용하면 제너레이터 함수가 됨
- 이터러블을 사용하면 for 루프의 다형성을 보장하는 강력한 추상화 가능

In [5]:
"""Clean Code in Python - Chapter 7: Using Generators

> Introduction to generators

"""
from _generate_data import PURCHASES_FILE, create_purchases_file
from log import logger


class PurchasesStats:
    def __init__(self, purchases):
        self.purchases = iter(purchases)
        self.min_price: float = None
        self.max_price: float = None
        self._total_purchases_price: float = 0.0
        self._total_purchases = 0
        self._initialize()

    def _initialize(self):
        try:
            first_value = next(self.purchases)
        except StopIteration:
            raise ValueError("no values provided")

        self.min_price = self.max_price = first_value
        self._update_avg(first_value)

    def process(self):
        for purchase_value in self.purchases:
            self._update_min(purchase_value)
            self._update_max(purchase_value)
            self._update_avg(purchase_value)
        return self

    def _update_min(self, new_value: float):
        if new_value < self.min_price:
            self.min_price = new_value

    def _update_max(self, new_value: float):
        if new_value > self.max_price:
            self.max_price = new_value

    @property
    def avg_price(self):
        return self._total_purchases_price / self._total_purchases

    def _update_avg(self, new_value: float):
        self._total_purchases_price += new_value
        self._total_purchases += 1

    def __str__(self):
        return (
            f"{self.__class__.__name__}({self.min_price}, "
            f"{self.max_price}, {self.avg_price})"
        )


# def _load_purchases(filename):
#     purchases = []
#     with open(filename) as f:
#         for line in f:
#             *_, price_raw = line.partition(",")
#             purchases.append(float(price_raw))

#     return purchases


def load_purchases(filename):
    with open(filename) as f:
        for line in f:
            *_, price_raw = line.partition(",")
            yield float(price_raw)


def main():
    create_purchases_file(PURCHASES_FILE)
    purchases = load_purchases(PURCHASES_FILE)
    stats = PurchasesStats(purchases).process()
    logger.info("Results: %s", stats)


if __name__ == "__main__":
    main()


Results: PurchasesStats(0.0, 999999.0, 499999.5)


In [6]:
load_purchases("file")

<generator object load_purchases at 0x00000268CE9BA7B0>

# 2. 이상적인 반복
## 관용적인 반복 코드

In [8]:
class NumberSequence:
    def __init__(self, start=0):
        self.current = start

    def next(self):
        current = self.current
        self.current += 1
        return current

seq = NumberSequence()

In [10]:
list(zip(NumberSequence(), "abcdef"))

TypeError: 'NumberSequence' object is not iterable

- iter 을 구현해 객체를 반복가능하게 만듬

In [11]:
class SequenceOfNumbers:
    def __init__(self, start=0):
        self.current = start

    def __next__(self):
        current = self.current
        self.current += 1
        return current

    def __iter__(self):
        return self
list(zip(SequenceOfNumbers(), "abcdef"))

[(0, 'a'), (1, 'b'), (2, 'c'), (3, 'd'), (4, 'e'), (5, 'f')]

- 제너레이터 사용하기

In [12]:
def sequence(start=0):
    while True:
        yield start
        start += 1

list(zip(sequence(), "abcdef"))

[(0, 'a'), (1, 'b'), (2, 'c'), (3, 'd'), (4, 'e'), (5, 'f')]

## 파이썬 이터레이터 패턴
- `__iter__` 와 `__next__` 매직 메서드를 구현한 객체

### 이터레이션 인터페이스
- `__iter__` : 이터러블, 반복을 지원하는 객체, for 루프에서 사용 가능
- `__next__`: 이터레이터 : 함수 호출 시 일련의 값에 대해 한 번에 하나씩만 어떻게 생성하는지 알고 있는 객체

### 이터러블이 가능한 시퀀스 객체
> 객체가 시퀀스여서 우연히 반복 가능할 수 있지만, 반복을 위한 객체를 디자인 할때는 iter 매직 메서드를 구현하라
- iter가 없어도 `__getitem__` 과 `__len__` 이 있으면 반복 가능
- INdexError 예외가 발생할 때까지 순서대로 값을 제공, 반복에 대한 중지를 알리는 역할

In [14]:
class MappedRange:
    """Apply a transformation to a range of numbers."""

    def __init__(self, transformation, start, end):
        self._transformation = transformation
        self._wrapped = range(start, end)

    def __getitem__(self, index):
        value = self._wrapped.__getitem__(index)
        result = self._transformation(value)
        logger.debug("Index %d: %s", index, result)
        return result

    def __len__(self):
        return len(self._wrapped)

mr = MappedRange(abs, -10, 5)

In [15]:
mr[0]

10

In [16]:
list(mr)

[10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0, 1, 2, 3, 4]

# 3. 코루틴(coroutine)

## 메인루틴과 서브 루틴 : 종속된  관계
![기본 루틴 동작](https://dojang.io/pluginfile.php/13976/mod_page/content/3/041001.png)

## 코루틴
- 메인루틴과 서브루틴이 서로 대등한 관계
- 진입점(entry point)가 여러개인 함수
  - 진입점 : 함수의 코드를 실행하는 지점
![코루틴 동작](https://dojang.io/pluginfile.php/13976/mod_page/content/3/041002.png)

In [17]:
def number_coroutine():
    while True:        # 코루틴을 계속 유지하기 위해 무한 루프 사용
        x = (yield)    # 코루틴 바깥에서 값을 받아옴, yield를 괄호로 묶어야 함
        print(x)
 
co = number_coroutine()
next(co)      # 코루틴 안의 yield까지 코드 실행(최초 실행)
 
co.send(1)    # 코루틴에 숫자 1을 보냄
co.send(2)    # 코루틴에 숫자 2을 보냄
co.send(3)    # 코루틴에 숫자 3을 보냄

1
2
3


![동작 방식](https://dojang.io/pluginfile.php/13976/mod_page/content/3/041003.png)